### 데이터 로드 및 shape 확인

In [ ]:
import zipfile
import pandas as pd
from pathlib import Path

# 파일 경로 지정
zip_path = Path("../data/movie.zip")

with zipfile.ZipFile(zip_path, 'r') as z:
    # 압축 파일 내 파일 목록 출력
    print(z.namelist()) 
    
    # CSV 파일 이름 추출
    csv_file = [f for f in z.namelist() if f.endswith('.csv')][0]
    
    # 세 번째 시도: 'latin-1' 인코딩 지정
    df = pd.read_csv(z.open(csv_file), encoding='latin-1', low_memory=False)
    
# Unnamed가 포함된 컬럼 이름 리스트를 얻는다.
unnamed_cols = [col for col in df.columns if 'Unnamed:' in col]

# 해당 컬럼들을 데이터프레임에서 제거
df = df.drop(columns=unnamed_cols)
# 데이터프레임 크기 출력
print(df.shape)

### 컬럼 제거

In [ ]:
# 제거할 컬럼 정의
drop_cols = [
    "status",
    "adult",
    "backdrop_path",
    "homepage",
    "imdb_id",
    "original_title",
    "tagline"
]

# 실제 제거
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

df.shape

### 이상치 제거

In [ ]:
import pandas as pd
import re

# vote_average 컬럼을 숫자형으로 변환 (오류 방지)
df["vote_average"] = pd.to_numeric(df["vote_average"], errors="coerce")
# vote_average: 0~10 범위 유지
df = df[(df["vote_average"] >= 0) & (df["vote_average"] <= 10)]

# vote_count 컬럼을 숫자형으로 변환
df["vote_count"] = pd.to_numeric(df["vote_count"], errors="coerce")
# vote_count: 0 이상만
df = df[df["vote_count"] >= 0]

# release_date: datetime 변환
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")

# release_date: 연도 범위 제한
df = df[
    (df["release_date"].dt.year >= 1870) &
    (df["release_date"].dt.year <= 2026) |
    (df["release_date"].isna())
]

# runtime 컬럼을 숫자형으로 변환
df["runtime"] = pd.to_numeric(df["runtime"], errors="coerce")
# runtime: 30분 미만 제거
df = df[df["runtime"] >= 30]

# revenue / budget 컬럼을 숫자형으로 변환
df["revenue"] = pd.to_numeric(df["revenue"], errors="coerce")
df["budget"] = pd.to_numeric(df["budget"], errors="coerce")

# revenue / budget: 1000 미만은 이상치로 처리
df.loc[df["revenue"] < 1000, "revenue"] = pd.NA
df.loc[df["budget"] < 1000, "budget"] = pd.NA

# original_language: 영문 2~3글자만 유지
pattern = re.compile(r"^[a-zA-Z]{2,3}$")
df = df[df["original_language"].astype(str).apply(lambda x: bool(pattern.match(x)))]

# vote_count가 5 이상인 영화만 포함
df.loc[df["vote_count"] == 0, "vote_average"] = pd.NA
df = df[df["vote_count"] >= 5]

df.shape

### 결측치 제거

In [ ]:
# title 결측치 제거
df = df.dropna(subset=["title"]).reset_index(drop=True)

df.shape

### 텍스트 데이터 정제

In [ ]:
import re
import pandas as pd

# 이모티콘 + 특수 유니코드 제거 패턴
emoji_pattern = re.compile(
    "["  
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map
    "\U0001F1E0-\U0001F1FF"  # flags
    "\U00002700-\U000027BF"  # dingbats
    "\U00002600-\U000026FF"  # misc symbols
    "\U00002B00-\U00002BFF"  # arrows & geometric shapes
    "\U0001FA70-\U0001FAFF"  # symbols
    "]+", 
    flags=re.UNICODE
)

# 제어문자 / 이상한 공백 제거
control_pattern = re.compile(r"[\r\n\t]+")

# 다중 공백 제거 패턴
multi_space = re.compile(r"\s+")

def clean_text(x):
    if pd.isna(x):
        return x
    x = str(x)
    x = emoji_pattern.sub("", x)
    x = control_pattern.sub(" ", x)
    x = multi_space.sub(" ", x)
    x = x.strip()
    return x

def clean_director_name(name):
    """
    이상한 감독 이름 필터링
    - 알파벳, 공백, 점, 하이픈만 허용
    - 너무 짧은 이름 제거
    """
    if pd.isna(name):
        return None
    
    name = str(name).strip()
    
    # 빈 문자열
    if len(name) == 0:
        return None
    
    # 너무 짧은 이름 (2글자 미만)
    if len(name) < 2:
        return None
    
    # 알파벳, 공백, 점, 하이픈만 허용
    if not re.match(r'^[a-zA-Z\s.\-]+$', name):
        return None
    
    return name

# title, overview 정제
df["title"] = df["title"].apply(clean_text)
df["overview"] = df["overview"].apply(clean_text)

# director 정제 (추가!)
df["director"] = df["director"].apply(clean_director_name)

### 데이터 저장

In [ ]:
# 정제된 데이터 저장
output_path = Path("tmdb_cleaned.csv")
df.to_csv(output_path, index=False, encoding="utf-8-sig")